In [1]:
import os, sys
import numpy as np
import matplotlib.pyplot as plt
import pickle
from copy import deepcopy
import CBcm

In [2]:
from scipy import stats

import iminuit

In [3]:
sys.path.append('/home/jp/projects/atm_nu/mceq_tysak/muon_fit/modules')

In [4]:
physics_params = ['le_pi+','le_pi-','le_K+','le_K-',
                  #'le_p','le_n',

                  'he_pi+','he_pi-', 'he_K+','he_K-',
                   'he_p','he_n',
                  
                  'vhe1_pi+', 'vhe1_pi-', 
                  #'vhe1_K+','vhe1_K-',
                   #'vhe1_p','vhe1_n',
                  
                  #'vhe2_pi+', 'vhe2_pi-', 
                  #'vhe2_K+','vhe2_K-',
                  # 'vhe2_p','vhe2_n',
                  
                  'vhe3_pi+', 'vhe3_pi-', 
                  'vhe3_K+','vhe3_K-',
                   'vhe3_p','vhe3_n',
                  #'scale',

                   'GSF_1','GSF_2',
                   'GSF_3','GSF_4',
                   'GSF_5','GSF_6']

In [5]:
import mufit
reload(mufit)
import mufit_plots as mfp
reload(mfp)

MUFIT: Using quickflux2!
MUFIT: Using quickflux2!
MufitPlots does not use the mceq flux evaluator directly
MufitPlots does not use the mceq flux evaluator directly


<module 'mufit_plots' from '/home/jp/projects/atm_nu/mceq_tysak/muon_fit/modules/mufit_plots.pyc'>

In [6]:
fitname = 'prd22_qf2_v19_fixGSF5'
filename = '/home/jp/projects/atm_nu/ddm_results/'+fitname+'.pkl'


In [7]:
fit = mufit.MuFit(datasets = ['bess',
                              'l3c', 
                              'opera', 
                              'minos',
                              'cms',
                              'deis',
                              'mutronratio',
                             'deisratio'],
                  vhe_relations = {},
                 flux_splines = 'ddmgsf-v19',)


/home/jp/projects/atm_nu/mceq_tysak/muon_fit/modules/resources/DDM_splines_20221115_newstyle_smarter.pkl
Excluded params removed from known_params.
No calibration file for ddmgsf-v19
Adjusting DEIS name
ddmgsf-v19
('Using errors in mode: ', 'mixed')
MUTRON ratio: adding ALL errors in quadrature
DEIS ratio: adding ALL errors in quadrature


In [8]:
my_flux_parameters = {
            # The fit only has power for GSF_1 and the scale parameter
            'GSF_1': [0., False, -4, 4, 0., 1], 
            'GSF_2': [0., False, -4, 4, 0., 1], 
            'GSF_3': [0., False, -4, 4, 0., 1], 
            'GSF_4': [0., False, -4, 4, 0., 1], 
            'GSF_5': [0., True, -4, 4, 0., 1], 
            'GSF_6': [0., False, -4, 4, 0., 1], 
            #'scale':[0., True, -4, 4, 0., 0.5], # Don't mention scale at all in new tables

            # LE - sensitivity except K-
            'le_pi+': [0.8, False, -10, 10, 0., 1],
            'le_pi-': [-0., False, -10, 10, 0., 1],
            'le_K+': [0., False, -10, 10, 0., 1], 
            'le_K-': [0., False, -10, 10, 0., 1], 

           
            # HE - sensitivity for all (increased errors for K by 4x)
            'he_pi+': [-0.08, False, -10, 10, 0., 1], 
            'he_pi-': [8, False, -10, 10, 0., 1], 
            'he_K+': [0., False, -10, 10, 0., 1], 
            'he_K-': [0., False, -10, 10, 0., 1], 
            'he_p':[0., False, -10, 10, 0., 1], 
            'he_n':[0., False, -10, 10, 0., 1], 

            # VHE1 - Little power over all, very weak for K+
            'vhe1_pi+':[-0.3, False, -5., 5.,  0.,  2.5], # Verified, power
            'vhe1_pi-':[0,False, -33,  33,    0,  10], # Verified, power
           
            
            # VHE2 - Very little power for K, p and n
            #'vhe2_pi+':[-0, False, -5., 5.,  0.,  3.], # Verified, fit has power
            #'vhe2_pi-':[0, False, -33,  33,    0,   14], # Verified, little power

            # VHE3 - Power for pi-, marginal for everyone else
            'vhe3_pi+':[-0, False, -5., 5.,  0.,  2.5], # Verified, NO POWER
            'vhe3_pi-':[0, False, -33,  33,    0,   10], # Verified, little power
            'vhe3_K+': [ 0., False, -19,  19,  0,  10], # Verified, NO POWER
            'vhe3_K-': [0., False, -25,  30, 0.,  20.], # Verified, NO POWER
            'vhe3_p':[0., False, -16, 16,  0., 6], # Verified, NO POWER
            'vhe3_n':[0., False, -6, 6,   0,  2.5], # Verified, NO POWER
        }

In [9]:
mysyst = deepcopy(fit.systematics)
mysyst.update({# Fixing DEIS because errors are included by default
               'deis_mscat':  [0.0, True, -3, 3],
               'deis_bfield':  [0.0, True, -3, 3],
               'deis_norm':  [0.0, True, -3, 3],

               # # Added for testing
#                    'amsratio_mc': 0., # np.random.uniform(-3,3),
#                    'amsratio_statistics':  0., #np.random.uniform(-3,3),
#                    'amsratio_protons':  0., #np.random.uniform(-3,3),

               # L3c (all)
               'l3c_align': 0.0042, #np.random.uniform(-3,3),
               'l3c_bfield': -3.30, #np.random.uniform(-3,3),
               'l3c_eloss': -2.92, #np.random.uniform(-3,3),
               'l3c_overb': -0.5, #np.random.uniform(-3,3),
               'l3c_normHor': -1.41, #np.random.uniform(-3,3),
               'l3c_normVert': -1.591, #np.random.uniform(-3,3),
               'l3c_pres': 1.352, #np.random.uniform(-3,3),

               'mn_escale': -1.59, #np.random.uniform(-3,3),

               # Bess
               'bess_align': -1.72, #np.random.uniform(-3,3),
               'bess_finres': 0.2, #np.random.uniform(-3,3),
               })

In [10]:
nfits = 1
counter = 0
bestfit = 1E9
defprior = 60
for ifit in range(nfits):
    print 'Internal fit ', ifit
    one_m, one_f = fit.run_fit(
        flux_parameters = my_flux_parameters,        
        exp_parameters = mysyst,
        fit_data = 'all',
        fit_mode = 'physics',
        use_priors = True,
        print_mode = False,
        dry_run = False,
        emin=5.,
        fake_data = False,
        )
    
    this_fit = fit.results['fmin']['fval']
    print 'This fit', this_fit
    if this_fit < bestfit:
        print 'Updated results for fit', this_fit, bestfit
        bestfit = deepcopy(this_fit)
        results = deepcopy(fit.results)
        m = one_m
        f = one_f

Internal fit  0


/home/jp/projects/atm_nu/mceq_tysak/muon_fit/modules/mufit.py:301: RuntimeWarning: invalid value encountered in divide
  phys_priors_mean)**2/phys_priors_err**2)[prior_bool]


Successful fit!
This fit 501.482515048
Updated results for fit 501.482515048 1000000000.0


In [11]:
free_params = 0
for one_param in fit.fit_keys:
    if not fit.minuit_settings['fix_'+one_param]:
        free_params += 1
#print(free_params, len(fit.fit_keys))

In [12]:
all_keys = results['params'].keys()
dof = fit.results['bins']-free_params
print 'Chi2',fit.results['chi2'], '/', dof, 'dof (approximate)'
p_value = 1-stats.chi2.cdf(fit.results['chi2'], dof)
print('P-value', p_value)
syst_keys = []
print '\t'.join(['Param', 'value','error'])
for one_param in physics_params:
    all_keys.remove(one_param)
    print '\t'.join([one_param, 
                     "%0.2f" % results['params'][one_param]['value'],
                    "%0.2f" % results['params'][one_param]['error']])
all_keys.sort()
for one_param in all_keys:
    print '\t'.join([one_param, 
                     "%0.2f" % results['params'][one_param]['value'],
                    "%0.2f" % results['params'][one_param]['error']])



Chi2 426.8375431790469 / 425 dof (approximate)
('P-value', 0.4658137194761608)
Param	value	error
le_pi+	-0.99	0.78
le_pi-	-1.98	0.37
le_K+	-0.66	0.76
le_K-	-0.06	0.99
he_pi+	-0.06	0.10
he_pi-	3.64	0.71
he_K+	0.26	0.98
he_K-	0.27	0.99
he_p	0.71	0.95
he_n	2.46	0.79
vhe1_pi+	-1.32	0.35
vhe1_pi-	-2.39	2.27
vhe3_pi+	-5.00	1.30
vhe3_pi-	-2.15	7.26
vhe3_K+	4.72	6.76
vhe3_K-	-21.70	28.37
vhe3_p	11.16	5.28
vhe3_n	3.00	2.27
GSF_1	1.08	0.90
GSF_2	0.37	0.99
GSF_3	0.11	0.99
GSF_4	0.60	0.99
GSF_5	0.00	0.50
GSF_6	0.53	0.98
bess_align	-2.26	0.67
bess_finres	0.20	0.83
deis_bfield	0.00	0.50
deis_mscat	0.00	0.50
deis_norm	0.00	0.50
l3c_align	-0.03	0.07
l3c_bfield	-3.71	0.78
l3c_eloss	-2.89	0.79
l3c_normHor	-1.38	0.55
l3c_normVert	-1.45	0.52
l3c_overb	-0.17	0.50
l3c_pres	1.46	0.48
mn_escale	-1.70	0.32


In [13]:
fit.results['flux_parameters'] = my_flux_parameters

In [14]:
fit.results['flux_tables']

'ddmgsf-v19'

In [15]:
pickle.dump(fit.results, open(filename, 'wb'))
print('Written', filename)

('Written', '/home/jp/projects/atm_nu/ddm_results/prd22_qf2_v19_fixGSF5.pkl')


In [16]:
m.matrix(correlation=True)

---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|              |        he_K-        he_K+       he_pi-       he_pi+       vhe3_n     vhe1_pi+      vhe3_K-      vhe3_K+     vhe1_pi-       vhe3_p       le_pi+       le_pi-        le_K+        le_K-        GSF_4        GSF_6        GSF_1        GSF_2        GSF_3     vhe3_pi+         he_p     vhe3_pi-         he_n   l3c_bfield    l3c_overb    mn_escale   bess_align l3c_normVert  l3c_normHor    l3c_eloss  bess_finres     l3c_pres    l3c_align |
---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|        he_K- |        1.000        0.018       -0.079        0.047        0.016        0.003       -0.050        0.020       -0.005       -0.000        0.001       -0.002        0.000        0.001       -0.001        0.001       -0.020       -0.001       -0.000       -0.000       -0.013        0.054       -0.026        0.005       -0.006        0.007        0.015        0.004       -0.001        0.003       -0.003       -0.001        0.003 |
|        he_K+ |        0.018        1.000        0.029       -0.282       -0.030       -0.008        0.098       -0.068        0.004        0.003       -0.013       -0.015       -0.014        0.000        0.001       -0.003       -0.011        0.001        0.000        0.000        0.006       -0.112       -0.016        0.027        0.022       -0.015        0.021        0.009        0.004        0.019        0.002        0.021        0.023 |
|       he_pi- |       -0.079        0.029        1.000        0.725       -0.056        0.283        0.021        0.043        0.293       -0.142        0.085        0.212        0.105        0.021       -0.020       -0.001       -0.290       -0.011       -0.003        0.001       -0.144        0.110       -0.481        0.033       -0.105        0.018        0.169        0.046        0.003        0.012       -0.043       -0.063       -0.024 |
|       he_pi+ |        0.047       -0.282        0.725        1.000       -0.110        0.367       -0.043        0.139        0.500       -0.227        0.050        0.109       -0.039        0.009       -0.006        0.014       -0.420       -0.003        0.003        0.001       -0.276        0.125       -0.120        0.034       -0.120        0.009        0.189        0.053        0.008        0.014       -0.049       -0.081       -0.031 |
|       vhe3_n |        0.016       -0.030       -0.056       -0.110        1.000       -0.107        0.091        0.001       -0.353       -0.127        0.038        0.095        0.050        0.004       -0.006       -0.007       -0.038       -0.004       -0.001        0.001       -0.024       -0.098       -0.119       -0.040       -0.006       -0.004       -0.058       -0.020       -0.010       -0.031        0.007       -0.003       -0.012 |
|     vhe1_pi+ |        0.003       -0.008        0.283        0.367       -0.107        1.000       -0.360       -0.378        0.879       -0.373       -0.004       -0.020        0.014       -0.006        0.001       -0.024       -0.642        0.007        0.002       -0.000        0.028        0.072        0.200        0.048        0.085       -0.010       -0.026        0.022        0.040        0.045        0.

In [17]:
fit.results['cov_params'] = m.list_of_vary_param()
ordered_matrix = np.zeros_like(fit.results['corr_matrix'])
# Need to reorder this business
syst_params = ['l3c_bfield','l3c_overb','l3c_normVert', 'bess_align','bess_finres',
               'l3c_normHor','l3c_eloss','l3c_pres','l3c_align', 'mn_escale']
syst_params.sort()
ordered_phys = np.zeros(len(physics_params))
ordered_syst = np.zeros(len(syst_params))
for ip, one_param in enumerate(physics_params):
    ordered_phys[ip] = fit.results['cov_params'].index(one_param)
for ip, one_param in enumerate(syst_params):
    ordered_syst[ip] = fit.results['cov_params'].index(one_param)
    
full_list = physics_params + syst_params
full_in = np.array(np.concatenate((ordered_phys, ordered_syst)),dtype=int)
for j, one_param in enumerate(full_list):
    for k in range(len(full_in)):
        ordered_matrix[j,k] = fit.results['corr_matrix'][full_in[j], full_in[k]]

ValueError: 'GSF_5' is not in list

In [ ]:
if not os.path.exists('/home/jp/projects/atm_nu/ddm_results/'+fitname):
    os.mkdir('/home/jp/projects/atm_nu/ddm_results/'+fitname)

In [ ]:
maxb = len(physics_params)
corr_matrix = deepcopy(ordered_matrix)

for i in range(corr_matrix.shape[0]):
    for j in range(corr_matrix.shape[0]):
        if i<j: corr_matrix[i,j]=0.

params = deepcopy(full_list)

# Adjust parameter names
for i, onep in enumerate(params):
    if 'he' in onep or 'K' in onep or 'pi' in onep:
        if 'K+' in onep:
            params[i] = onep.replace('_K+', ' $K^+$')
        elif 'K-' in onep:
            params[i] = onep.replace('_K-', ' $K^-$')
        elif 'pi+' in onep:
            params[i] = onep.replace('_pi+', ' $\pi^+$')
        elif 'pi-' in onep:
            params[i] = onep.replace('_pi-', ' $\pi^-$')
        elif '_p' in params[i]:
            params[i] = onep.replace('_p', ' $p$')
        elif '_n' in params[i]:
            params[i] = onep.replace('_n', ' $n$')    
    elif 'l3c' in params[i]:
        params[i] = 'L3+c ' + params[i].lstrip('l3c_')
    elif 'mn' in params[i]:
        params[i] ='MINOS '  + params[i].lstrip('mn_')
    elif 'bess' in params[i]:
        params[i] = 'BessTeV ' + params[i].lstrip('bess_')
    elif 'GSF' in params[i]:
        params[i] = onep.replace('_', '')
        
    #if 'align' in params[i]:
    #    params[i] = params[i].replace('align','alignment')
    if 'bfield' in params[i]:
       params[i] = params[i].replace('bfield','$B$ field')
    if 'eloss' in params[i]:
       params[i] = params[i].replace('eloss','$E$ loss')
    if 'pres' in params[i]:
       params[i] = params[i].replace('pres','$p_{res}$')
    if 'escale' in params[i]:
       params[i] = params[i].replace('escale','$E$ scale')
    
fig = plt.figure(figsize=(12*0.7,10*0.7))
ax = fig.add_subplot(111)
xmarks = np.arange(0.5, corr_matrix.shape[0], 1.)
plt.pcolor(corr_matrix[:maxb,:maxb], cmap='RdBu', vmin=-1, vmax=1)
plt.xticks(xmarks, params[:maxb], rotation=90)
plt.yticks(xmarks, params[:maxb])
plt.ylim(0,maxb)
plt.xlim(0,maxb)
ax.set_ylim(ax.get_ylim()[::-1])
ax.xaxis.tick_top()
plt.colorbar()
plt.tight_layout()



fig.savefig('/home/jp/projects/atm_nu/ddm_results/'+fitname+'/corr_phys_noscale_qf2.png', dpi=300)
fig.savefig('/home/jp/projects/atm_nu/ddm_results/'+fitname+'/corr_phys_noscale_qf2.pdf')


In [ ]:
maxb = None
corr_matrix = deepcopy(ordered_matrix)

for i in range(corr_matrix.shape[0]):
    for j in range(corr_matrix.shape[0]):
        if i<j: corr_matrix[i,j]=0.

params = deepcopy(full_list)

# Adjust parameter names
for i, onep in enumerate(params):
    if 'he' in onep or 'K' in onep or 'pi' in onep:
        if 'K+' in onep:
            params[i] = onep.replace('_K+', ' $K^+$')
        elif 'K-' in onep:
            params[i] = onep.replace('_K-', ' $K^-$')
        elif 'pi+' in onep:
            params[i] = onep.replace('_pi+', ' $\pi^+$')
        elif 'pi-' in onep:
            params[i] = onep.replace('_pi-', ' $\pi^-$')
        elif '_p' in params[i]:
            params[i] = onep.replace('_p', ' $p$')
        elif '_n' in params[i]:
            params[i] = onep.replace('_n', ' $n$')    
    elif 'l3c' in params[i]:
        params[i] = 'L3+c ' + params[i].lstrip('l3c_')
    elif 'mn' in params[i]:
        params[i] ='MINOS '  + params[i].lstrip('mn_')
    elif 'bess' in params[i]:
        params[i] = 'BessTeV ' + params[i].lstrip('bess_')
    elif 'GSF' in params[i]:
        params[i] = onep.replace('_', '')
        
    #if 'align' in params[i]:
    #    params[i] = params[i].replace('align','alignment')
    if 'bfield' in params[i]:
       params[i] = params[i].replace('bfield','$B$ field')
    if 'eloss' in params[i]:
       params[i] = params[i].replace('eloss','$E$ loss')
    if 'pres' in params[i]:
       params[i] = params[i].replace('pres','$p_{res}$')
    if 'escale' in params[i]:
       params[i] = params[i].replace('escale','$E$ scale')
    
fig = plt.figure(figsize=(14,10))
ax = fig.add_subplot(111)
xmarks = np.arange(0.5, corr_matrix.shape[0], 1.)
plt.pcolor(corr_matrix[:maxb,:maxb], cmap='RdBu', vmin=-1, vmax=1)
plt.xticks(xmarks, params[:maxb], rotation=90)
plt.yticks(xmarks, params[:maxb])
plt.ylim(0,maxb)
plt.xlim(0,maxb)
ax.set_ylim(ax.get_ylim()[::-1])
ax.xaxis.tick_top()
plt.colorbar()
#fig.savefig('/home/jp/projects/atm_nu/ddm_results/prd22_results_may9_v12_final/corr_full.png', dpi=300)
#fig.savefig('/home/jp/projects/atm_nu/ddm_results/prd22_results_may9_v12_final/corr_full.pdf')


In [ ]:
inb = 27
maxb = None
    
fig = plt.figure(figsize=(14,10))
ax = fig.add_subplot(111)
xmarks = np.arange(0.5, corr_matrix[inb:maxb,inb:maxb].shape[0], 1.)
plt.pcolor(corr_matrix[inb:maxb,inb:maxb], cmap='RdBu', vmin=-1, vmax=1)
plt.xticks(xmarks, params[inb:maxb], rotation=90)
plt.yticks(xmarks, params[inb:maxb])
plt.ylim(0,maxb)
plt.xlim(0,maxb)
ax.set_ylim(ax.get_ylim()[::-1])
ax.xaxis.tick_top()
plt.colorbar()
#fig.savefig('/home/jp/projects/atm_nu/ddm_results/prd22_results_may9_v12_final/corr_syst.png', dpi=300)
#fig.savefig('/home/jp/projects/atm_nu/ddm_results/prd22_results_may9_v12_final/corr_syst.pdf')


In [ ]:
# Plots of the stored fit fluxes, ratios directly - does not evaluate splines again
reload(mfp)
mfp.plot_comparison(fit, pullflag=10.,
                   emin=5)

In [ ]:
reload(mfp)

In [ ]:
# Plots of the stored fit fluxes, ratios directly - does not evaluate splines again
f1 = mfp.plotICRC21(fit, data_prefit = True)
f1.savefig('/home/jp/projects/atm_nu/ddm_results/muon_results/flux_vert_postmufix_noscale_qf2smart.pdf')

In [ ]:
reload(mfp)
f2 = mfp.plotICRC21(fit, data= 'ratio',data_prefit = True)
f2.savefig('/home/jp/projects/atm_nu/ddm_results/muon_results/ratio_vert_postmufix_noscale_qf2smart.pdf')

In [ ]:
# Plots of the stored fit fluxes, ratios directly - does not evaluate splines again
reload(mfp)
f3 = mfp.plotICRC21(fit, data='deis', data_prefit = True)
f3.savefig('/home/jp/projects/atm_nu/ddm_results/muon_results/flux_deis_postmufix_noscale_qf2smart.pdf')

In [ ]:
reload(mfp)
f1 = mfp.plotPulls(mufit=fit, pull_scale=20, log_pulls=False
                  )